# The Revised Simplex Method

While solving a problem with simplex method, successive iterations are obtained by using row operations. This requires storing the entire table in the memory of the computer, which may not be feasible for very large problems. Luckily, it is really not necessary to calculate the entire table during each iteration. The only information needed in moving from one table to the next is

1. The $c_j - Z_j$ row to dertermine the non-basic variable that enters the basis
2. The pivote column
3. The current basic variables and their values (right-hand side constants) to determine the minimum positive ratio and thereby to determine the basic variable that leaves the basis.

In the revised simplex method, the above information is directly obtained from the original equations of the problem by making use of the current basis matrix $\mathbf{B}$ and its inverse $\mathbf{B}^{-1}$. This method will now be illustrated with the help of some examples.

## Example

$$\max Z = 6x_1 + 3x_2 + 4x_3 - 2x_4 + x_5$$

\begin{align*}
2x_1 + 3x_2 + 3x_3 + x_4 & = 10\\
x_1 + 2x_2 + x_3 + x_5 & = 8\\[5mm]
x_1, x_2, x_3, x_4, x_5 & \geq 0
\end{align*}

In [1]:
import numpy as np

In [2]:
%run "../auxfunc/simplex_algorithm.ipynb"

In [66]:
cj = np.array([6, 3, 4, -2, 1], dtype=float)

A = np.array([
    [2, 3, 3, 1, 0],
    [1, 2, 1, 0, 1],
], dtype=float)

b = np.array([10, 8], dtype=float)

In [114]:
solutions_primal, zvalues, lastrows_z, cbindexes = simplex(matrix=A, rhs=b, z=cj, numxvars=3)

Iteration 1.  x4 --> x1
[[ 1.   1.5  1.5  0.5  0. ]
 [ 0.   0.5 -0.5 -0.5  1. ]] 

cb [6. 1.] 

Solution [5. 0. 0. 0. 3.] 	Z: 33.00 

Optimal solution found in 1 iterations


In [165]:
cb_index = [3,4]
cb = cj[cb_index]

In [166]:
B = A[:, cb_index]
print(B)

[[1. 0.]
 [0. 1.]]


In [167]:
Binv = np.linalg.inv(B)
print("\n", Binv)


 [[1. 0.]
 [0. 1.]]


In [168]:
body = Binv.dot(A)
print(body)
rhs = Binv.dot(b)
print(rhs)

[[2. 3. 3. 1. 0.]
 [1. 2. 1. 0. 1.]]
[10.  8.]


In [169]:
pimult = cb.dot(Binv)
zj = pimult.dot(A)
print(pimult)

[-2.  1.]


In [170]:
ner = cj - pimult.dot(A)
print(ner)

[9. 7. 9. 0. 0.]


In [171]:
entering = np.array(ner).argmax()
print(entering)

key_column = A[:, entering]
ratios = np.divide(rhs, key_column, out=np.full_like(b, np.inf), where=key_column>0)
leaving = ratios.argmin()
print(leaving)

0


In [173]:
cb_index = [0,4]
B = A[:, cb_index]
Binv = np.linalg.inv(B)
print("Binv\n", Binv)

Binv
 [[ 0.5  0. ]
 [-0.5  1. ]]


In [174]:
cb = cj[cb_index]

In [175]:
body = Binv.dot(A)
print(body)
rhs = Binv.dot(rhs)
print("rhs", rhs)

[[ 1.   1.5  1.5  0.5  0. ]
 [ 0.   0.5 -0.5 -0.5  1. ]]
rhs [5. 3.]


In [176]:
pimult = cb.dot(Binv)
zj = pimult.dot(A)
print(pimult)

[2.5 1. ]


In [177]:
ner = cj - zj
print(ner)

[ 0.  -6.5 -4.5 -4.5  0. ]


In [178]:
print(np.array(cb))
print(rhs)

[6. 1.]
[5. 3.]


In [ ]:
def simplex_revised(matrix, rhs, z, numxvars, direction=1):
    '''The Revised Simplex algorithm to solve linear programming problems
    
    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side
    
    rhs: numpy ndarray
        Right-hand side vector
    
    numxvars: int
        Number of x variables
        
    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    
    Returns
    ---------
    solutions: np.ndarray
        Array of solutions for every iteration when an optimal solution was found
    favlues: np.ndarray
        Array of the objective function values in every iteration
    lastrows: np.ndarray
        Array of the last two rows (zj, cj - zj) in the optimal solution table
    ''' 
    matrix = np.array(matrix, dtype=float)
    rhs = np.array(rhs)
    z = np.array(z)
    
    num_rows, num_cols = matrix.shape
    labels = [f"x{i + 1}" for i in range(num_cols)]
    onecols = np.where(matrix == 1)[1]
    cb_index = onecols[onecols >= numxvars]
    cb = z[cb_index]
                       
    zj = cb.dot(matrix)
                       
    net_evaluation = direction * (z - zj)
    
    solutions = []
    fvalues = []
    cbindexlist = []
    iteration = 0
    
    B = A[:, cb_index]
    Binv = np.linalg.inv(B)
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)
        entering_label = labels[entering]

        key_col = matrix[ : , entering]
        ratios = np.divide(rhs, key_col, out=np.full_like(rhs, np.inf), where=key_col>0)
        leaving = ratios.argmin()   # leaving variables (index)
        leaving_label = labels[cb_index[leaving]]     
        
        pivot = matrix[leaving, entering]
        
        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot
        
        for i in range(num_rows):
            if i == leaving:
                continue
            factor = matrix[i, entering]
            matrix[i] = -factor * matrix[leaving] + matrix[i]
            rhs[i] = -factor * rhs[leaving] + rhs[i]
        
        cb_index[leaving] = entering
        cb = z[cb_index]
        
        zj = cb.dot(matrix)
        
        net_evaluation = direction * (z - zj)

        solution[cb_index] = rhs  # basics
        
        iteration += 1
        print(f"Iteration {iteration}.  {leaving_label} --> {entering_label}")
        print(matrix,  "\n")
        print("cb", cb, "\n")
        print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}", "\n")
        
        solutions.append(solution)
        fvalues.append(cb.dot(rhs))
        cbindexlist.append(cb_index)
        if np.all(net_evaluation <= 0):
            print(f"Optimal solution found in {iteration} iterations")
    return np.array(solutions), fvalues, np.vstack((zj, net_evaluation)), np.array(cbindexlist)